# Basic Example

This notebook shows a basic example of how a langchain agent can be set up to use the jupyter notebook tools.


In [6]:
# Install the package first, in this example we'll just install from the editable source
# in the repository
# !pip install -e ../src/

In [7]:
# The module contains standard python logging so if you want to see how the tool is used
# without using an LLM observation framework this is one way to do it.
import logging

# Set up root logger
logging.basicConfig(
    level=logging.WARN, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Set the log level for the jupyter_tool logger to DEBUG or above
logging.getLogger("jupyter_tool").setLevel(logging.DEBUG)

In [8]:
# Include any LLM observability framework you might want. There is no additional observability
# inside of the module, we leverage langchain methods for all LLM observability.

# from langfuse.callback import CallbackHandler

# langfuse_handler = CallbackHandler(
#     secret_key="...",
#     public_key="...",
#     host="http://172.20.0.7:3000",
# )

In [9]:
# Each function in the module is a unique tool which can be called by the agent.
from jupyter_tool import (
    load_notebook,
    create_notebook,
    execute_cell,
    delete_cell,
    update_cell,
    list_cells,
    create_cell,
)

# For this example we'll use AWS Bedrock with Anthropic's Claude model. Different models
# will use tools well (or poorly), and this is largely a function of the model's
# capabilities around whether they were pretrained to engage in multiturn conversations with
# function calls.
from langchain_aws.chat_models import ChatBedrockConverse

model = ChatBedrockConverse(
    model="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name="us-west-2",
)

# Make a list of the tools you want to give the agent access to
tools = [
    load_notebook,
    create_notebook,
    execute_cell,
    delete_cell,
    update_cell,
    list_cells,
    create_cell,
]

# In this basic example we'll use the langgraph prebuilt react agent to execute the tools.
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)

result = langgraph_agent_executor.invoke(
    {
        "messages": [
            (
                "user",
                """Create a new jupyter notebook. In the first cell write a small python function sort_asc(lst) that
                sorts the elements of a list in ascending order and execute it. In the second cell, write a small python
                function sort_desc(lst) that sorts the elements of a list in descending order and execute it. In the third cell, write a
                the following and execute it: sort_asc(sort_desc([3, 2, 1, 4, 5])). Provide the output of the third cell.
                """,
            )
        ]
    },
    # Uncomment if using an observability framework
    # config={"callbacks": [langfuse_handler]},
)
print(result["messages"][-1].content)

2025-02-26 15:49:43,838 - jupyter_tool.jupyter - DEBUG - Creating new empty notebook
2025-02-26 15:49:44,389 - jupyter_tool.jupyter - DEBUG - Created new notebook token: 9ad5eade-b1ec-404e-9f69-2d4c2204cfd0
2025-02-26 15:49:46,869 - jupyter_tool.jupyter - DEBUG - Creating new code cell in notebook 9ad5eade-b1ec-404e-9f69-2d4c2204cfd0
2025-02-26 15:49:46,872 - jupyter_tool.jupyter - DEBUG - Created new cell with ID: 16052fb0-9218-43e7-bdab-465ae0d5d2c3
2025-02-26 15:49:49,993 - jupyter_tool.jupyter - DEBUG - Executing cell 16052fb0-9218-43e7-bdab-465ae0d5d2c3 for notebook 9ad5eade-b1ec-404e-9f69-2d4c2204cfd0
2025-02-26 15:49:49,994 - jupyter_tool.jupyter - DEBUG - Attempting to execute cell 16052fb0-9218-43e7-bdab-465ae0d5d2c3 at index 0
2025-02-26 15:49:50,009 - jupyter_tool.jupyter - DEBUG - Cell 16052fb0-9218-43e7-bdab-465ae0d5d2c3 at index 0 executed successfully
2025-02-26 15:49:52,298 - jupyter_tool.jupyter - DEBUG - Creating new code cell in notebook 9ad5eade-b1ec-404e-9f69-2d4c2

Let me explain what happened:
1. First, `sort_desc([3, 2, 1, 4, 5])` sorts the list in descending order, resulting in `[5, 4, 3, 2, 1]`
2. Then, `sort_asc([5, 4, 3, 2, 1])` sorts this list in ascending order
3. The final output is `[1, 2, 3, 4, 5]`

The output of the third cell is `[1, 2, 3, 4, 5]`.
